---
# ADAS - Queries and Analytics on Video

The data scientist now wants to focus on video frames in which trains were detected.

The metadata for the video stream produced by the object detection job is loaded into a Pandas dataframe.
This dataframe is used to allow the data scientist to view any image stored in SDP.
They can also filter, sort, and aggregate the dataframe using methods that data scientists are familar with.

---

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import IPython
import cv2
import itertools
import numpy as np
import pandas as pd
import json
import base64
import datetime
import time
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoPlayer
from matplotlib import pyplot as plt
from copy import copy
import os

imp.reload(video);

In [2]:
pd.options.display.max_rows = 1000

### Define Pravega stream parameters

In [3]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
stream = 'object-detector-output-video'

### Initialize connection to Pravega GRPC Gateway

In [4]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### Load timestamp index
This is an index from timestamp to begin stream cut, end stream cut, and event pointer.
This index is maintained by the job in [index_builder.ipynb](index_builder.ipynb).
The video player uses the event pointer to fetch an individual frame. It uses the end stream cut to play from a particular frame.

In [5]:
indexed_stream = IndexedStream(pravega_client, scope, stream)

In [6]:
%%time
indexed_stream.load_index(limit=10000)

load_index: Reading index stream object-detector-output-video-index
CPU times: user 1.54 s, sys: 725 ms, total: 2.26 s
Wall time: 7.77 s


#### Show number of records in the index, first and last index records

In [7]:
len(indexed_stream.index_df)

10000

In [8]:
indexed_stream.index_df.iloc[[0,-1]].T

timestamp,2020-04-21 18:55:05.344000+00:00,2020-04-21 19:23:55.985000+00:00
camera,2,0
ssrc,0,0
frameNumber,0,645
hash,None,None
tags,None,None
kittiSensorReadings,None,None
recognitions,"[{'id': 6, 'title': 'car', 'confidence': 0.372...","[{'id': 14, 'title': 'person', 'confidence': 0..."
sourceEventPointer,{'eventPointerBytes': 'AAABAAAAADAAImV4YW1wbGV...,{'eventPointerBytes': 'AAABAAAAADAAImV4YW1wbGV...
chunkIndex,0,0
finalChunkIndex,0,0


#### Clean recognitions

In [9]:
def clean_recognitions(recognitions):
    return ','.join(np.unique([r['title'] for r in recognitions]))
indexed_stream.index_df['recog'] = indexed_stream.index_df.recognitions.apply(clean_recognitions)

### Video player with entire stream

In [10]:
#player = VideoPlayer(stream=indexed_stream).interact()

### Filtered and sorted player

In [ ]:
#indexed_stream.index_df.camera.value_counts()

In [ ]:
#indexed_stream.index_df.recog.value_counts()

In [12]:
pd.DataFrame(indexed_stream.index_df.groupby(['camera','recog']).size()).unstack().fillna('-').T.head(20)

camera                                 0   1  2   3    4    5   6  7
  recog                                                             
0                                      5  40  8  15    -  100  37  3
  aeroplane,car,person                 -   -  3   -    -    -   -  -
  bicycle                             23   -  3   -    -    -   -  -
  bicycle,boat,person                  2   -  -   -    -    -   -  -
  bicycle,bottle                       2   -  -   -    -    -   -  -
  bicycle,bottle,chair,person          1   -  -   -    -    -   -  -
  bicycle,bottle,person                6   -  -   -    -    -   -  -
  bicycle,bus                          -   -  -   -   32    -   -  -
  bicycle,bus,car                      -   -  -   -   16    -   -  -
  bicycle,bus,car,person               -   -  -   -   24    -   -  -
  bicycle,bus,motorbike,person         4   -  -   -   16    -   -  -
  bicycle,bus,person                   8   -  -   -  112    -   -  -
  bicycle,car                          8   -  -   -    8   10   -  3
  bicycle,car,chair,person             6   -  -   -    -    -   -  -
  bicycle,car,chair,person,train       2   -  -   -    -    -   -  -
  bicycle,car,chair,person,tvmonitor   2   -  -   -    -    -   -  -
  bicycle,car,motorbike,person         8   -  -   -    -    -   -  -
  bicycle,car,motorbike,person,train   2   -  -   -    -    -   -  -
  bicycle,car,motorbike,pottedplant    1   -  -   -    -    -   -  -
  bicycle,car,person                  35   -  6   -   63    -   4  3

In [ ]:
# df = indexed_stream.index_df.copy()
#df = df[df.camera==0]
#df = df[df.frameNumber.between(10,20)]
#df = df[df.recog!='']
# df = df[df.recog.str.contains('tvmonitor')]
# df = df[df.recog.str.contains('person')]
#df = df[df.recog.str.contains('pottedplant')]
#df = df[df.recog.str.contains('bus,person')]
# df = df[df.recog.str.contains('bus')]
# df = df[df.recog.str.contains('train')]
# df = df[df.recog.str.contains('chair')]
#df = df[df.recog.str.contains('dog')]
#df = df[df.recog.str.contains('sofa')]
# df.camera.value_counts()

In [13]:
df = indexed_stream.index_df.copy()
df = df[df.recog.str.contains('bus')]
df.camera.value_counts()

4    461
5    380
6    368
0    219
7    133
3    106
2    102
1     84
Name: camera, dtype: int64

In [15]:
filtered_stream = copy(indexed_stream)
filtered_stream.index_df = df
VideoPlayer(stream=filtered_stream).interact()